In [1]:
from spatialoperations import SpatialOperations
from spatialoperations.config import SpatialOpsConfig


config = SpatialOpsConfig(
    root="s3://cl-lake-test/sids-test-tough-topobathy",
    dx=0.00026977476123492776,
    epsg=4326,
    bounds=[-179.99, -50, 179.99, 50],
    chunk_size=5000,
)
so = SpatialOperations(config=config)
so

In [2]:
import rioxarray as rxr
import geopandas as gpd
import os
from tqdm import tqdm
from glob import glob
from spatialoperations.types.xrtypes import RasterLike
import numpy as np

countries = [
    "FSM_10",
    "FSM_05",
    "FSM_08",
    "FSM_21",
    "FSM_20",
    "FSM_18",
    "FSM_11",
    "KIR_09"
]
ROOT = "/cccr-lab/001_projects/002_nbs-adapt/003_regions_study_units/All_Regions_Marina/02_Polygons_SIDS_20250701/02_ACA_Benthic_Bathymetry_Data/"

paths = []
for country in countries:
    paths.extend(glob(os.path.join(ROOT, f'{country}*/Bathymetry---composite-depth/bathymetry_0.tif')))

def safe_get_region(path):
    try:
        ds = rxr.open_rasterio(path).isel(band=0)
        init_crs = ds.rio.crs
        ds = ds * -.01 # convert to meters
        ds = ds.where(ds < 0, np.nan)
        ds.rio.write_crs(init_crs, inplace=True)
        return RasterLike(ds, name=path.split('/')[-3])
    except Exception as e:
        return None
        
ACA = {
    path.split('/')[-3].split('-')[0]: safe_get_region(path)
    for idx, path in tqdm(enumerate(paths))
}

ACA = {
    k: v for k, v in ACA.items() if v is not None
}

ACA

0it [00:00, ?it/s]

5it [00:04,  1.03it/s]


{'FSM_10': RasterLike(_data=<xarray.DataArray (y: 6102, x: 11001)> Size: 537MB
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]])
 Coordinates:
     band         int64 8B 1
   * x            (x) float64 88kB 149.5 149.5 149.5 149.5 ... 150.5 150.5 150.5
   * y            (y) float64 49kB 9.053 9.053 9.053 9.053 ... 8.505 8.505 8.505
     spatial_ref  int64 8B 0, name='FSM_10-20250701210124'),
 'FSM_05': RasterLike(_data=<xarray.DataArray (y: 3051, x: 11500)> Size: 281MB
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]])
 Coordin

In [3]:
# import hvplot.xarray

# for name, raster in ACA.items():
#     print(f"Plotting {name}")
#     display(raster._data.hvplot.image(x='x', y='y', rasterize=True, cmap='viridis', title=name))
#     break


In [5]:
# so.ro.zarr_manager.root_group.create_group("topobathy/bathymetry")

In [6]:
from spatialoperations import rasterops
from spatialoperations.rasterops import intake
from spatialoperations import compute

jlc = compute.JoblibCompute(n_jobs=20)

so.ro.intake.import_multi_raster_to_zarr_with_prep(
    ACA, 
    "topobathy/bathymetry", 
    output_path="ACA",
    compute=jlc,
)

100%|█████████████████████████████████████████| 4/4 [00:00<00:00, 44979.13it/s]


In [8]:
so.ro.zarr_manager.chunks_initialized(
    "ACA",
    "topobathy/bathymetry"
)

[(30, 244),
 (30, 245),
 (30, 246),
 (31, 241),
 (31, 242),
 (32, 251),
 (36, 261),
 (36, 262),
 (37, 261),
 (37, 262)]